# Kittel Data

In [8]:
from music21 import *
import copy
import collections
import itertools

kittelBundle = corpus.search('Kittel-24-Worksheets')
melBundle = corpus.search('melodic')

In [9]:
# pass in the function that extracts what we want, and the bundle of scores we want to analyse, coalesces the data
def getData(func, bundle):
    data = []
    for b in bundle:
        d = func(converter.parse(b))
        data.extend(d)
    counter=collections.Counter(data)
    for k,v in sorted(counter.items(), key=lambda x: -x[1]):
        print(k,',',v)

# m is the melody as a chord, b is the bass as a chord
# if either are empty return "R" to represent rest, otherwise return the interval
def getHarmony(m, b):
    if len(m) == 0 or len(b) == 0:
        return "R"
    else:
        return interval.Interval(m[0], b[0]).semiSimpleName
    
def getInterval(m, b):
    if len(m) == 0 or len(b) == 0:
        return "R"
    else:
        return interval.Interval(m[0], b[0]).directedName

## 2 Half Note Segments

### Melodic

In [42]:
def melody2HN(k):
    graceNotes = []
    for n in k.recurse().notes:
        if n.duration.isGrace:
            graceNotes.append(n)

    # remove grace notes  
    for grace in graceNotes:
        grace.activeSite.remove(grace)

    k_ = copy.deepcopy(k)
    k_c = k_.chordify()
    data = []
    for i in range(1, len(k_.parts[0].getElementsByClass('Measure')) + 1):
        k_mel = k_.measure(i).parts[0].asTimespans()
        mel = []
        off = k_.parts[0].measure(i).recurse().notes[0].offset

        # starting window
        mel.append(interval.Interval(k_mel.getVerticalityAt(off).toChord()[0],
                                     k_mel.getVerticalityAt(off + 2).toChord()[0]).directedName)
        data.append(mel.copy())
        off += 2

        # slide the window
        while off <= k_c.measure(i).notes[-1].offset - 2:
            mel.pop(0)
            mel.append(interval.Interval(k_mel.getVerticalityAt(off).toChord()[0],
                                         k_mel.getVerticalityAt(off + 2).toChord()[0]).directedName)
            data.append(mel.copy())
            off += 2
    data = map(lambda x: tuple(x), data)
    return data

In [43]:
getData(melody2HN, melBundle)

('M-2',) , 213
('P1',) , 158
('M2',) , 141
('m-2',) , 87
('m2',) , 69
('m-3',) , 37
('m3',) , 26
('P4',) , 23
('M-3',) , 15
('M3',) , 14
('P5',) , 6
('P-4',) , 5
('P-5',) , 2


### Melodic Phrase Ending

In [46]:
def melody2HNPhraseEnd(k):
    graceNotes = []
    for n in k.recurse().notes:
        if n.duration.isGrace:
            graceNotes.append(n)

    # remove grace notes  
    for grace in graceNotes:
        grace.activeSite.remove(grace)

    k_ = copy.deepcopy(k)
    k_c = k_.chordify()
    data = []
    for i in range(1, len(k_.parts[0].getElementsByClass('Measure')) + 1):
        k_mel = k_.measure(i).parts[0].asTimespans()
        mel = []
        off = k_c.measure(i).notes[-1].offset - 2

        # starting window
        mel.append(interval.Interval(k_mel.getVerticalityAt(off).toChord()[0],
                                     k_mel.getVerticalityAt(off + 2).toChord()[0]).directedName)
        data.append(mel.copy())
    data = map(lambda x: tuple(x), data)
    return data

In [47]:
getData(melody2HNPhraseEnd, melBundle)

('M-2',) , 64
('m2',) , 21
('M2',) , 15
('m-2',) , 7
('P1',) , 6
('m-3',) , 4
('M-3',) , 2
('m3',) , 1


### Full

In [48]:
# the func to be passed into getData, this is a sliding window of size two half notes in the melody,
# three quarter notes in the bass
def full2HN(k):
    graceNotes = []
    for n in k.recurse().notes:
        if n.duration.isGrace:
            graceNotes.append(n)

    # remove grace notes  
    for grace in graceNotes:
        grace.activeSite.remove(grace)

    k_ = copy.deepcopy(k)
    k_c = k_.chordify()
    data = []
    for i in range(1, len(k_.parts[0].getElementsByClass('Measure')) + 1):
        k_mel = k_.measure(i).parts[0].asTimespans()
        k_bass = k_.measure(i).parts[1].asTimespans()
        mel = []
        bass = []
        off = k_.parts[0].measure(i).recurse().notes[0].offset

        # starting window
        har_1 = getHarmony(k_mel.getVerticalityAt(off).toChord(),
                           k_bass.getVerticalityAt(off).toChord())
        mel.append(getInterval(k_mel.getVerticalityAt(off).toChord(),
                               k_mel.getVerticalityAt(off + 2).toChord()))
        bass.append(getInterval(k_bass.getVerticalityAt(off).toChord(),
                                k_bass.getVerticalityAt(off + 2).toChord()))
        har_2 = getHarmony(k_mel.getVerticalityAt(off + 2).toChord(),
                           k_bass.getVerticalityAt(off + 2).toChord())
        data.append((har_1, mel.copy(), bass.copy(), har_2))
        off += 2

        # slide the window
        while off <= k_c.measure(i).notes[-1].offset - 2:
            mel.pop(0)
            bass.pop(0)
            har_1 = getHarmony(k_mel.getVerticalityAt(off).toChord(),
                           k_bass.getVerticalityAt(off).toChord())
            mel.append(getInterval(k_mel.getVerticalityAt(off).toChord(),
                                   k_mel.getVerticalityAt(off + 2).toChord()))
            bass.append(getInterval(k_bass.getVerticalityAt(off).toChord(),
                                    k_bass.getVerticalityAt(off + 2).toChord()))
            har_2 = getHarmony(k_mel.getVerticalityAt(off + 2).toChord(),
                           k_bass.getVerticalityAt(off + 2).toChord())
            data.append((har_1, mel.copy(), bass.copy(), har_2))
            off += 2
    data = map(lambda x: (x[0], tuple(x[1]), tuple(x[2]), x[3]), data)
    return data

In [49]:
getData(full2HN, kittelBundle)

('P5', ('M-2',), ('P-5',), 'P8') , 246
('M6', ('P1',), ('M2',), 'P5') , 153
('P5', ('P1',), ('P1',), 'P5') , 118
('m3', ('M2',), ('m2',), 'M3') , 116
('P5', ('M2',), ('P4',), 'M3') , 114
('m6', ('m-3',), ('m2',), 'M3') , 105
('M6', ('M2',), ('m-2',), 'P8') , 104
('m3', ('M-2',), ('m2',), 'P8') , 104
('P8', ('M-2',), ('m3',), 'P5') , 95
('M6', ('m2',), ('M-2',), 'P8') , 95
('m6', ('P1',), ('m2',), 'P5') , 92
('M6', ('M-2',), ('m-2',), 'm6') , 89
('M3', ('M-2',), ('P-4',), 'P5') , 88
('M3', ('m2',), ('P-5',), 'P8') , 88
('P5', ('M-2',), ('P4',), 'P8') , 79
('M3', ('M-2',), ('P5',), 'P5') , 76
('m6', ('m-2',), ('M-2',), 'M6') , 70
('M6', ('M-2',), ('M-2',), 'M6') , 69
('P8', ('M2',), ('M-2',), 'M3') , 66
('M2', ('P1',), ('m-2',), 'm3') , 64
('A4', ('m2',), ('m-2',), 'm6') , 61
('m3', ('m-2',), ('M2',), 'P8') , 61
('m6', ('M2',), ('M-2',), 'P8') , 58
('P5', ('M-2',), ('M2',), 'm3') , 57
('d5', ('M-2',), ('m2',), 'm3') , 57
('P5', ('M-2',), ('m-3',), 'm6') , 55
('M3', ('m2',), ('m-3',), 'm6

### Full Phrase Ending

In [50]:
# the func to be passed into getData, this is a sliding window of size two half notes in the melody,
# three quarter notes in the bass
def full2HNPhraseEnd(k):
    graceNotes = []
    for n in k.recurse().notes:
        if n.duration.isGrace:
            graceNotes.append(n)

    # remove grace notes  
    for grace in graceNotes:
        grace.activeSite.remove(grace)

    k_ = copy.deepcopy(k)
    k_c = k_.chordify()
    data = []
    for i in range(1, len(k_.parts[0].getElementsByClass('Measure')) + 1):
        k_mel = k_.measure(i).parts[0].asTimespans()
        k_bass = k_.measure(i).parts[1].asTimespans()
        mel = []
        bass = []
        off = k_c.measure(i).notes[-1].offset - 2

        # starting window
        har_1 = getHarmony(k_mel.getVerticalityAt(off).toChord(),
                           k_bass.getVerticalityAt(off).toChord())
        mel.append(getInterval(k_mel.getVerticalityAt(off).toChord(),
                               k_mel.getVerticalityAt(off + 2).toChord()))
        bass.append(getInterval(k_bass.getVerticalityAt(off).toChord(),
                                k_bass.getVerticalityAt(off + 2).toChord()))
        har_2 = getHarmony(k_mel.getVerticalityAt(off + 2).toChord(),
                           k_bass.getVerticalityAt(off + 2).toChord())
        data.append((har_1, mel.copy(), bass.copy(), har_2))
    data = map(lambda x: (x[0], tuple(x[1]), tuple(x[2]), x[3]), data)
    return data

In [51]:
getData(full2HNPhraseEnd, kittelBundle)

('P5', ('M-2',), ('P-5',), 'P8') , 244
('M3', ('m2',), ('P-5',), 'P8') , 83
('P5', ('M-2',), ('P4',), 'P8') , 77
('M3', ('m2',), ('P4',), 'P8') , 27
('P5', ('M2',), ('P-5',), 'M3') , 27
('P8', ('M-2',), ('m3',), 'P5') , 25
('m3', ('M-2',), ('m2',), 'P8') , 25
('M6', ('m2',), ('M-2',), 'P8') , 25
('M3', ('M-2',), ('P5',), 'P5') , 23
('M3', ('M-2',), ('P-4',), 'P5') , 23
('P5', ('M2',), ('P4',), 'M3') , 23
('M6', ('M2',), ('m-2',), 'P8') , 18
('P5', ('M-2',), ('m-3',), 'm6') , 16
('m6', ('P1',), ('M-3',), 'P8') , 15
('m3', ('M2',), ('m2',), 'M3') , 13
('P5', ('P1',), ('P-4',), 'P8') , 12
('P8', ('m-3',), ('P-5',), 'M3') , 11
('M6', ('M-2',), ('m-2',), 'm6') , 10
('d5', ('M-2',), ('m2',), 'm3') , 10
('P8', ('m-3',), ('P4',), 'M3') , 10
('P5', ('P1',), ('P1',), 'P5') , 10
('m7', ('m-2',), ('P4',), 'M3') , 9
('m3', ('P1',), ('m3',), 'P8') , 9
('d5', ('m-2',), ('m2',), 'M3') , 8
('M3', ('m2',), ('m-3',), 'm6') , 8
('m7', ('m-2',), ('P-5',), 'M3') , 7
('P8', ('M2',), ('M-2',), 'M3') , 7
('m6'

## 3 Half Note Segments

### 3HN Melodic

In [52]:
def melody3HN(k):
    graceNotes = []
    for n in k.recurse().notes:
        if n.duration.isGrace:
            graceNotes.append(n)

    # remove grace notes  
    for grace in graceNotes:
        grace.activeSite.remove(grace)

    k_ = copy.deepcopy(k)
    k_c = k_.chordify()
    data = []
    for i in range(1, len(k_.parts[0].getElementsByClass('Measure')) + 1):
        k_mel = k_.measure(i).parts[0].asTimespans()
        mel = []
        off = k_.parts[0].measure(i).recurse().notes[0].offset

        # starting window
        mel.append(interval.Interval(k_mel.getVerticalityAt(off).toChord()[0],
                                     k_mel.getVerticalityAt(off + 2).toChord()[0]).directedName)
        mel.append(interval.Interval(k_mel.getVerticalityAt(off + 2).toChord()[0],
                                     k_mel.getVerticalityAt(off + 4).toChord()[0]).directedName)
        data.append(mel.copy())
        off += 2

        # slide the window
        while off <= k_c.measure(i).notes[-1].offset - 4:
            mel.pop(0)
            mel.append(interval.Interval(k_mel.getVerticalityAt(off + 2).toChord()[0],
                                         k_mel.getVerticalityAt(off + 4).toChord()[0]).directedName)
            data.append(mel.copy())
            off += 2
    data = map(lambda x: tuple(x), data)
    return data

In [53]:
getData(melody3HN, melBundle)

('P1', 'M-2') , 62
('m-2', 'M-2') , 52
('M-2', 'P1') , 41
('M-2', 'M-2') , 38
('M2', 'M2') , 33
('M2', 'm2') , 33
('M-2', 'm-2') , 33
('P1', 'P1') , 29
('M2', 'M-2') , 22
('P1', 'M2') , 22
('M2', 'P1') , 21
('P1', 'm-2') , 20
('M-2', 'M2') , 17
('m-2', 'm2') , 17
('m2', 'm-2') , 16
('m2', 'M2') , 15
('m-3', 'M-2') , 10
('P1', 'm-3') , 9
('m2', 'P1') , 9
('M-2', 'P4') , 9
('M-3', 'M2') , 7
('m3', 'M-2') , 7
('m3', 'M2') , 7
('M2', 'm3') , 6
('M3', 'M-2') , 6
('M-2', 'm-3') , 6
('m-3', 'M2') , 6
('m3', 'm-2') , 6
('P4', 'P1') , 6
('m-2', 'P1') , 6
('m-3', 'M-3') , 5
('m-3', 'P1') , 5
('P4', 'M2') , 5
('M2', 'm-3') , 5
('m2', 'm-3') , 4
('P4', 'M-2') , 4
('M-2', 'm3') , 4
('M3', 'm2') , 4
('P4', 'm-3') , 3
('P1', 'M-3') , 3
('m-3', 'm3') , 3
('P4', 'm-2') , 3
('M2', 'M-3') , 3
('m-3', 'm2') , 3
('m3', 'P-5') , 2
('M2', 'P4') , 2
('m-2', 'M-3') , 2
('P5', 'M2') , 2
('P5', 'P1') , 2
('m3', 'm-3') , 2
('M3', 'm3') , 2
('P4', 'P-4') , 2
('P-4', 'M2') , 2
('P5', 'm-3') , 2
('P1', 'P4') , 2
('m

### 3HN Phrase Ending

In [54]:
def melody3HNPhraseEnd(k):
    graceNotes = []
    for n in k.recurse().notes:
        if n.duration.isGrace:
            graceNotes.append(n)

    # remove grace notes  
    for grace in graceNotes:
        grace.activeSite.remove(grace)

    k_ = copy.deepcopy(k)
    k_c = k_.chordify()
    data = []
    for i in range(1, len(k_.parts[0].getElementsByClass('Measure')) + 1):
        k_mel = k_.measure(i).parts[0].asTimespans()
        mel = []
        off = k_c.measure(i).notes[-1].offset - 4

        # starting window
        mel.append(interval.Interval(k_mel.getVerticalityAt(off).toChord()[0],
                                     k_mel.getVerticalityAt(off + 2).toChord()[0]).directedName)
        mel.append(interval.Interval(k_mel.getVerticalityAt(off + 2).toChord()[0],
                                     k_mel.getVerticalityAt(off + 4).toChord()[0]).directedName)
        data.append(mel.copy())
    data = map(lambda x: tuple(x), data)
    return data

In [55]:
getData(melody3HNPhraseEnd, melBundle)

('P1', 'M-2') , 45
('m-2', 'm2') , 13
('M2', 'm2') , 8
('M-2', 'M-2') , 7
('m-2', 'M-2') , 7
('P1', 'M2') , 6
('P1', 'm-2') , 5
('M-2', 'M2') , 4
('P1', 'P1') , 4
('P1', 'm-3') , 3
('m-3', 'M-2') , 3
('M2', 'M2') , 3
('M-3', 'M2') , 2
('m-3', 'P1') , 1
('P1', 'M-3') , 1
('M-2', 'm-3') , 1
('M3', 'm3') , 1
('M2', 'M-2') , 1
('m-3', 'M-3') , 1
('M-2', 'm-2') , 1
('M-3', 'm-2') , 1
('M3', 'M-2') , 1
('M-2', 'P1') , 1


### 3HN Full

In [56]:
def full3HN(k):
    graceNotes = []
    for n in k.recurse().notes:
        if n.duration.isGrace:
            graceNotes.append(n)

    # remove grace notes  
    for grace in graceNotes:
        grace.activeSite.remove(grace)

    k_ = copy.deepcopy(k)
    k_c = k_.chordify()
    data = []
    for i in range(1, len(k_.parts[0].getElementsByClass('Measure')) + 1):
        k_mel = k_.measure(i).parts[0].asTimespans()
        k_bass = k_.measure(i).parts[1].asTimespans()
        mel = []
        bass = []
        off = k_.parts[0].measure(i).recurse().notes[0].offset

        # starting window
        har_1 = getHarmony(k_mel.getVerticalityAt(off).toChord(),
                           k_bass.getVerticalityAt(off).toChord())
        mel.append(getInterval(k_mel.getVerticalityAt(off).toChord(),
                               k_mel.getVerticalityAt(off + 2).toChord()))
        mel.append(getInterval(k_mel.getVerticalityAt(off + 2).toChord(),
                               k_mel.getVerticalityAt(off + 4).toChord()))
        bass.append(getInterval(k_bass.getVerticalityAt(off).toChord(),
                                k_bass.getVerticalityAt(off + 2).toChord()))
        bass.append(getInterval(k_bass.getVerticalityAt(off + 2).toChord(),
                                k_bass.getVerticalityAt(off + 4).toChord()))
        har_2 = getHarmony(k_mel.getVerticalityAt(off + 4).toChord(),
                           k_bass.getVerticalityAt(off + 4).toChord())
        data.append((har_1, mel.copy(), bass.copy(), har_2))
        off += 2

        # slide the window
        while off <= k_c.measure(i).notes[-1].offset - 4:
            mel.pop(0)
            bass.pop(0)
            har_1 = getHarmony(k_mel.getVerticalityAt(off).toChord(),
                           k_bass.getVerticalityAt(off).toChord())
            mel.append(getInterval(k_mel.getVerticalityAt(off + 2).toChord(),
                                   k_mel.getVerticalityAt(off + 4).toChord()))
            bass.append(getInterval(k_bass.getVerticalityAt(off + 2).toChord(),
                                    k_bass.getVerticalityAt(off + 4).toChord()))
            har_2 = getHarmony(k_mel.getVerticalityAt(off + 4).toChord(),
                           k_bass.getVerticalityAt(off + 4).toChord())
            data.append((har_1, mel.copy(), bass.copy(), har_2))
            off += 2
    data = map(lambda x: (x[0], tuple(x[1]), tuple(x[2]), x[3]), data)
    return data

In [57]:
getData(full3HN, kittelBundle)

('M6', ('P1', 'M-2'), ('M2', 'P-5'), 'P8') , 121
('P5', ('P1', 'M-2'), ('P1', 'P-5'), 'P8') , 62
('m6', ('M2', 'M2'), ('M-2', 'M-2'), 'M3') , 38
('P5', ('P1', 'M-2'), ('P-8', 'P4'), 'P8') , 37
('M3', ('M2', 'm2'), ('m-3', 'M-2'), 'P8') , 35
('M6', ('M2', 'm2'), ('m-2', 'M-2'), 'm3') , 31
('P5', ('m-2', 'm2'), ('M2', 'P-5'), 'P8') , 29
('M3', ('M-2', 'P1'), ('P5', 'P1'), 'P5') , 26
('M3', ('M-2', 'P1'), ('P-5', 'M2'), 'P5') , 25
('M3', ('M-2', 'M-2'), ('M2', 'M2'), 'm6') , 25
('m6', ('m-2', 'M-2'), ('M-2', 'M-2'), 'M6') , 25
('m3', ('m-2', 'M-2'), ('M2', 'm3'), 'P5') , 24
('M6', ('M-2', 'm-2'), ('m-2', 'M-2'), 'M6') , 24
('m3', ('m-2', 'M-2'), ('M2', 'm2'), 'M6') , 23
('M6', ('M-2', 'M-2'), ('P-8', 'P4'), 'P8') , 22
('M3', ('M-2', 'P1'), ('P1', 'm-2'), 'm3') , 22
('M6', ('M-2', 'M-2'), ('M-2', 'm-2'), 'm6') , 22
('P8', ('M-2', 'P1'), ('m2', 'M2'), 'P5') , 22
('M2', ('P1', 'M-2'), ('m-2', 'm2'), 'P8') , 21
('P5', ('M2', 'm2'), ('P4', 'P-5'), 'P8') , 21
('m6', ('m-2', 'M-2'), ('m3', 'P-4'

('P8', ('M-2', 'm-2'), ('P5', 'P1'), 'M2') , 1
('m3', ('M3', 'M-2'), ('m3', 'd-4'), 'd5') , 1
('M3', ('m3', 'm-2'), ('m-2', 'M-2'), 'M6') , 1
('M3', ('P1', 'm2'), ('m-3', 'P4'), 'm3') , 1
('P5', ('m2', 'm-2'), ('P4', 'P-4'), 'P5') , 1
('A4', ('m2', 'M-2'), ('M-2', 'm-2'), 'm6') , 1
('M6', ('M-2', 'm-2'), ('m-2', 'm-2'), 'm6') , 1
('m6', ('m-2', 'P1'), ('m-2', 'm2'), 'P5') , 1
('P8', ('P1', 'M2'), ('m6', 'P1'), 'A4') , 1
('M6', ('m3', 'm-2'), ('P4', 'm-3'), 'M6') , 1
('m6', ('m-2', 'M-2'), ('P1', 'm-3'), 'm6') , 1
('A4', ('m2', 'M-2'), ('m-2', 'M-2'), 'm6') , 1
('m6', ('M-2', 'm-2'), ('M-2', 'P1'), 'P5') , 1
('m3', ('M3', 'M-2'), ('P1', 'P1'), 'P4') , 1
('P5', ('M-2', 'M-2'), ('P1', 'P1'), 'm3') , 1
('P4', ('M-2', 'm-2'), ('P1', 'R'), 'R') , 1
('m3', ('m-2', 'P1'), ('R', 'R'), 'P5') , 1
('R', ('P1', 'M-2'), ('R', 'P-5'), 'P8') , 1
('P8', ('M-2', 'M-2'), ('m2', 'P-4'), 'P8') , 1
('M6', ('M-2', 'm-2'), ('P-4', 'M2'), 'M6') , 1
('P8', ('m-2', 'P1'), ('M2', 'M2'), 'P5') , 1
('P8', ('P1', 'M

### 3HN Full Phrase End

In [58]:
def full3HNPhraseEnd(k):
    graceNotes = []
    for n in k.recurse().notes:
        if n.duration.isGrace:
            graceNotes.append(n)

    # remove grace notes  
    for grace in graceNotes:
        grace.activeSite.remove(grace)

    k_ = copy.deepcopy(k)
    k_c = k_.chordify()
    data = []
    for i in range(1, len(k_.parts[0].getElementsByClass('Measure')) + 1):
        k_mel = k_.measure(i).parts[0].asTimespans()
        k_bass = k_.measure(i).parts[1].asTimespans()
        mel = []
        bass = []
        off = k_c.measure(i).notes[-1].offset - 4

        # starting window
        har_1 = getHarmony(k_mel.getVerticalityAt(off).toChord(),
                           k_bass.getVerticalityAt(off).toChord())
        mel.append(getInterval(k_mel.getVerticalityAt(off).toChord(),
                               k_mel.getVerticalityAt(off + 2).toChord()))
        mel.append(getInterval(k_mel.getVerticalityAt(off + 2).toChord(),
                               k_mel.getVerticalityAt(off + 4).toChord()))
        bass.append(getInterval(k_bass.getVerticalityAt(off).toChord(),
                                k_bass.getVerticalityAt(off + 2).toChord()))
        bass.append(getInterval(k_bass.getVerticalityAt(off + 2).toChord(),
                                k_bass.getVerticalityAt(off + 4).toChord()))
        har_2 = getHarmony(k_mel.getVerticalityAt(off + 4).toChord(),
                           k_bass.getVerticalityAt(off + 4).toChord())
        data.append((har_1, mel.copy(), bass.copy(), har_2))
    data = map(lambda x: (x[0], tuple(x[1]), tuple(x[2]), x[3]), data)
    return data

In [59]:
getData(full3HNPhraseEnd, kittelBundle)

('M6', ('P1', 'M-2'), ('M2', 'P-5'), 'P8') , 121
('P5', ('P1', 'M-2'), ('P1', 'P-5'), 'P8') , 62
('P5', ('P1', 'M-2'), ('P-8', 'P4'), 'P8') , 37
('P5', ('m-2', 'm2'), ('M2', 'P-5'), 'P8') , 29
('M6', ('M-2', 'M-2'), ('P-8', 'P4'), 'P8') , 22
('P5', ('M2', 'm2'), ('P4', 'P-5'), 'P8') , 20
('M2', ('P1', 'M-2'), ('m-2', 'm2'), 'P8') , 19
('P4', ('m-2', 'm2'), ('P-8', 'P4'), 'P8') , 16
('m6', ('P1', 'M-2'), ('m2', 'P-5'), 'P8') , 15
('M3', ('M2', 'm2'), ('m-3', 'M-2'), 'P8') , 12
('m7', ('m-2', 'm2'), ('P4', 'P-5'), 'P8') , 12
('M6', ('P1', 'M2'), ('M2', 'P-5'), 'M3') , 10
('M3', ('M2', 'm2'), ('M2', 'P-5'), 'P8') , 9
('P8', ('P1', 'M-2'), ('P4', 'P-5'), 'P8') , 9
('M6', ('M-2', 'M2'), ('P-8', 'P4'), 'M3') , 8
('P5', ('M-2', 'P1'), ('m-3', 'M-3'), 'P8') , 8
('P5', ('P1', 'M-2'), ('P1', 'P4'), 'P8') , 7
('M3', ('M-2', 'M-2'), ('P-4', 'P-5'), 'P8') , 7
('m6', ('m-3', 'M-2'), ('m2', 'P-4'), 'P5') , 7
('m6', ('P1', 'P1'), ('m2', 'P-4'), 'P8') , 7
('P8', ('P1', 'm-3'), ('P1', 'P4'), 'M3') , 7
(